In [2]:
import pandas as pd

In [3]:
original_confirmed=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',error_bad_lines=False)
original_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',error_bad_lines=False)
original_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',error_bad_lines=False)


In [4]:
original_confirmed.columns[4:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20',
       '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20',
       '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20',
       '4/13/20', '4/14/20', '4/15

## Merging DataFrame


In [5]:
dates = original_confirmed.columns[4:]

original_confirmed_tran = original_confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

original_deaths_trans = original_deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

original_recovered_trans = original_recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')


print(original_confirmed_tran.shape)
print(original_deaths_trans.shape)
print(original_recovered_trans.shape)

(25608, 6)
(25608, 6)
(24250, 6)


In [6]:
df_final = pd.merge(left=original_confirmed_tran, right=original_deaths_trans, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
df_final = pd.merge(left=df_final, right=original_recovered_trans, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

df_final.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0.0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0.0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0.0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0.0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0.0


In [7]:
df_final.isnull().sum()

Province/State    17654
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          2716
dtype: int64

In [8]:
# Fill null with 0 and change Recovered to int
df_final['Recovered'] = df_final['Recovered'].fillna(0)
df_final['Recovered'] = df_final['Recovered'].astype('int')
df_final.isna().sum()

Province/State    17654
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [9]:
# Check data types
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25608 entries, 0 to 25607
Data columns (total 8 columns):
Province/State    7954 non-null object
Country/Region    25608 non-null object
Lat               25608 non-null float64
Long              25608 non-null float64
Date              25608 non-null object
Confirmed         25608 non-null int64
Deaths            25608 non-null int64
Recovered         25608 non-null int32
dtypes: float64(2), int32(1), int64(2), object(3)
memory usage: 1.7+ MB


In [10]:
# Change date to datetime
df_final.Date=pd.to_datetime(df_final['Date'])

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25608 entries, 0 to 25607
Data columns (total 8 columns):
Province/State    7954 non-null object
Country/Region    25608 non-null object
Lat               25608 non-null float64
Long              25608 non-null float64
Date              25608 non-null datetime64[ns]
Confirmed         25608 non-null int64
Deaths            25608 non-null int64
Recovered         25608 non-null int32
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(2)
memory usage: 1.7+ MB


### Save data on the google drive

In [12]:
# save file
df_final.to_csv('covid_19_complete.csv', index=False)